In [1]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, ALL_TIMES,interpline, CoordPair, xy_to_ll, ll_to_xy)
import pandas as pd
import numpy as np
import os
import shutil
import re
import netCDF4 as nc
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter,LatitudeFormatter
import matplotlib.ticker as mticker
import cartopy.crs as ccrs                                               
import cartopy.feature as cf
import matplotlib.pyplot as plt
import glob
from scipy.stats import norm
from scipy import stats
import seaborn as sns
import shapefile
import geopandas as gpd
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
#mask out seas-1620

ds_lu_1620 = xr.open_dataset('wrfout_d04_2011-06-23_11:00:00')
lu_1620 = ds_lu_1620['LU_INDEX']

lu_1620 = lu_1620[0,:,:]

del lu_1620.coords['XTIME']

lu_1620_df = lu_1620.to_dataframe().reset_index()

In [3]:
#mask out seas-2040s

ds_lu_2040s = xr.open_dataset('wrfout_d04_2040-06-18_23:00:00')
lu_2040s = ds_lu_2040s['LU_INDEX']

lu_2040s = lu_2040s[0,:,:]

del lu_2040s.coords['XTIME']

lu_2040s_df = lu_2040s.to_dataframe().reset_index()

# 2010s

In [4]:
ds1 = xr.open_dataset('ssp245_t2_1120_era5_corrected_dailymax.nc')
t2_1120_ssp245_dailymax = ds1['ssp245_t2_1120_era5_corrected_dailymax']

t2_1120_ssp245_dailymax_df = t2_1120_ssp245_dailymax.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_1120_dailymax_thres = (t2_1120_ssp245_dailymax_df[t2_1120_ssp245_dailymax_df['ssp245_t2_1120_era5_corrected_dailymax']>=33]).reset_index(drop=True)

In [5]:
t2_1120_dailymax_thres = t2_1120_dailymax_thres.sort_values(['XLAT','XLONG','Time'], ascending=True)


t2_1120_dailymax_thres_latlon = t2_1120_dailymax_thres.groupby(['XLAT','XLONG'])


t2_1120_dailymax_thres['auxi_col'] = t2_1120_dailymax_thres_latlon['Time'].rank()


t2_1120_dailymax_thres['date_sub'] = t2_1120_dailymax_thres['Time'] - pd.to_timedelta(t2_1120_dailymax_thres['auxi_col'],unit='d')

In [6]:
hw_meantemp_1120 = t2_1120_dailymax_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub'])['ssp245_t2_1120_era5_corrected_dailymax'].mean().reset_index()

In [7]:
hw_1120 = t2_1120_dailymax_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub']).count().reset_index()
hw_1120 = hw_1120.rename(columns={'auxi_col':'concecutive_hot_days_num'})

hw_1120 = hw_1120[hw_1120['concecutive_hot_days_num']>=3]

del hw_1120['Time']
del hw_1120['LU_INDEX']
del hw_1120['ssp245_t2_1120_era5_corrected_dailymax']

In [8]:
hw_1120 = pd.merge(hw_1120,hw_meantemp_1120,on=['XLAT','XLONG','south_north','west_east','date_sub'],how="left")

hw_1120 = hw_1120.groupby(['XLAT','XLONG','south_north','west_east'])['ssp245_t2_1120_era5_corrected_dailymax'].mean().reset_index()

In [9]:
#mask out the sea

hw_1120 = pd.merge(hw_1120,lu_1620_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

hw_1120['ssp245_t2_1120_era5_corrected_dailymax'] = hw_1120['ssp245_t2_1120_era5_corrected_dailymax'].fillna(0)

hw_1120.loc[hw_1120.LU_INDEX == 16, 'ssp245_t2_1120_era5_corrected_dailymax'] = 9999

hw_1120 = hw_1120.rename(columns={"ssp245_t2_1120_era5_corrected_dailymax": "hwt"})

In [10]:
hw_1120.to_csv('HWT_1120.csv',index=False,columns=['hwt','XLAT','XLONG'])

hw_1120.to_csv('HWT_1120_withgrids.csv',index=False,columns=['XLAT','XLONG','hwt','south_north','west_east'])

# 2040s

In [11]:
ds1 = xr.open_dataset('ssp245_t2_4049_era5_corrected_dailymax.nc')
t2_4049_ssp245_dailymax = ds1['ssp245_t2_4049_era5_corrected_dailymax']

t2_4049_ssp245_dailymax_df = t2_4049_ssp245_dailymax.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_4049_dailymax_thres = (t2_4049_ssp245_dailymax_df[t2_4049_ssp245_dailymax_df['ssp245_t2_4049_era5_corrected_dailymax']>=33]).reset_index(drop=True)

In [12]:
t2_4049_dailymax_thres = t2_4049_dailymax_thres.sort_values(['XLAT','XLONG','Time'], ascending=True)


t2_4049_dailymax_thres_latlon = t2_4049_dailymax_thres.groupby(['XLAT','XLONG'])

t2_4049_dailymax_thres['auxi_col'] = t2_4049_dailymax_thres_latlon['Time'].rank()


t2_4049_dailymax_thres['date_sub'] = t2_4049_dailymax_thres['Time'] - pd.to_timedelta(t2_4049_dailymax_thres['auxi_col'],unit='d')

In [13]:
hw_meantemp_4049 = t2_4049_dailymax_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub'])['ssp245_t2_4049_era5_corrected_dailymax'].mean().reset_index()

hw_4049 = t2_4049_dailymax_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub']).count().reset_index()
hw_4049 = hw_4049.rename(columns={'auxi_col':'concecutive_hot_days_num'})

hw_4049 = hw_4049[hw_4049['concecutive_hot_days_num']>=3]

del hw_4049['Time']
del hw_4049['LU_INDEX']
del hw_4049['ssp245_t2_4049_era5_corrected_dailymax']

hw_4049 = pd.merge(hw_4049,hw_meantemp_4049,on=['XLAT','XLONG','south_north','west_east','date_sub'],how="left")

hw_4049 = hw_4049.groupby(['XLAT','XLONG','south_north','west_east'])['ssp245_t2_4049_era5_corrected_dailymax'].mean().reset_index()

In [14]:
#mask out the sea

hw_4049 = pd.merge(hw_4049,lu_2040s_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

hw_4049['ssp245_t2_4049_era5_corrected_dailymax'] = hw_4049['ssp245_t2_4049_era5_corrected_dailymax'].fillna(0)

hw_4049.loc[hw_4049.LU_INDEX == 16, 'ssp245_t2_4049_era5_corrected_dailymax'] = 9999

hw_4049 = hw_4049.rename(columns={"ssp245_t2_4049_era5_corrected_dailymax": "hwt"})

In [15]:
hw_4049.to_csv('HWT_4049.csv',index=False,columns=['hwt','XLAT','XLONG'])

hw_4049.to_csv('HWT_4049_withgrids.csv',index=False,columns=['XLAT','XLONG','hwt','south_north','west_east'])

# 2090s

In [16]:
ds1 = xr.open_dataset('ssp245_t2_9099_era5_corrected_dailymax.nc')
t2_9099_ssp245_dailymax = ds1['ssp245_t2_9099_era5_corrected_dailymax']

t2_9099_ssp245_dailymax_df = t2_9099_ssp245_dailymax.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_9099_dailymax_thres = (t2_9099_ssp245_dailymax_df[t2_9099_ssp245_dailymax_df['ssp245_t2_9099_era5_corrected_dailymax']>=33]).reset_index(drop=True)

In [17]:
t2_9099_dailymax_thres = t2_9099_dailymax_thres.sort_values(['XLAT','XLONG','Time'], ascending=True)


t2_9099_dailymax_thres_latlon = t2_9099_dailymax_thres.groupby(['XLAT','XLONG'])


t2_9099_dailymax_thres['auxi_col'] = t2_9099_dailymax_thres_latlon['Time'].rank()


t2_9099_dailymax_thres['date_sub'] = t2_9099_dailymax_thres['Time'] - pd.to_timedelta(t2_9099_dailymax_thres['auxi_col'],unit='d')

In [18]:
hw_meantemp_9099 = t2_9099_dailymax_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub'])['ssp245_t2_9099_era5_corrected_dailymax'].mean().reset_index()

hw_9099 = t2_9099_dailymax_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub']).count().reset_index()
hw_9099 = hw_9099.rename(columns={'auxi_col':'concecutive_hot_days_num'})

hw_9099 = hw_9099[hw_9099['concecutive_hot_days_num']>=3]

del hw_9099['Time']
del hw_9099['LU_INDEX']
del hw_9099['ssp245_t2_9099_era5_corrected_dailymax']

hw_9099 = pd.merge(hw_9099,hw_meantemp_9099,on=['XLAT','XLONG','south_north','west_east','date_sub'],how="left")

hw_9099 = hw_9099.groupby(['XLAT','XLONG','south_north','west_east'])['ssp245_t2_9099_era5_corrected_dailymax'].mean().reset_index()

In [19]:
#mask out the sea

hw_9099 = pd.merge(hw_9099,lu_2040s_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

hw_9099['ssp245_t2_9099_era5_corrected_dailymax'] = hw_9099['ssp245_t2_9099_era5_corrected_dailymax'].fillna(0)

hw_9099.loc[hw_9099.LU_INDEX == 16, 'ssp245_t2_9099_era5_corrected_dailymax'] = 9999

hw_9099 = hw_9099.rename(columns={"ssp245_t2_9099_era5_corrected_dailymax": "hwt"})

In [20]:
hw_9099.to_csv('HWT_9099.csv',index=False,columns=['hwt','XLAT','XLONG'])

hw_9099.to_csv('HWT_9099_withgrids.csv',index=False,columns=['XLAT','XLONG','hwt','south_north','west_east'])

# diff

In [21]:
hw_1120 = pd.read_csv('HWT_1120_withgrids.csv')

hw_1120 = hw_1120.replace(9999,np.nan)

hw_1120 = hw_1120.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [22]:
hw_4049 = pd.read_csv('HWT_4049_withgrids.csv')

hw_4049 = hw_4049.replace(9999,np.nan)

hw_4049 = hw_4049.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [23]:
hw_9099 = pd.read_csv('HWT_9099_withgrids.csv')

hw_9099 = hw_9099.replace(9999,np.nan)

hw_9099 = hw_9099.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [24]:
#test

print((hw_1120['west_east']-hw_4049['west_east']).min())
print((hw_1120['west_east']-hw_4049['west_east']).max())
print((hw_1120['south_north']-hw_4049['south_north']).max())
print((hw_1120['south_north']-hw_4049['south_north']).min())
print((hw_4049['west_east']-hw_9099['west_east']).max())
print((hw_4049['west_east']-hw_9099['west_east']).min())
print((hw_4049['south_north']-hw_9099['south_north']).max())
print((hw_4049['south_north']-hw_9099['south_north']).min())

0
0
0
0
0
0
0
0


In [25]:
hw_diff_2040s = hw_1120.copy()
hw_diff_2090s = hw_1120.copy()

In [26]:
hw_diff_2040s['diff'] = hw_4049['hwt']-hw_1120['hwt']

hw_diff_2090s['diff'] = hw_9099['hwt']-hw_1120['hwt']

In [27]:
hw_diff_2040s['diff'] = hw_diff_2040s['diff'].fillna(9999)

hw_diff_2090s['diff'] = hw_diff_2090s['diff'].fillna(9999)

In [28]:
hw_diff_2040s.to_csv('HWT_diff_2040s.csv')
hw_diff_2090s.to_csv('HWT_diff_2090s.csv')